<a href="https://colab.research.google.com/github/MarpuriSathvik/ML-HW3/blob/main/HW3_ML_TASK1_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [2]:
data = pd.read_csv('/content/drive/MyDrive/MG/data.csv')
labels = pd.read_csv('/content/drive/MyDrive/MG/label.csv',names=['label'],header=None)

In [3]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [5]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [6]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [7]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [8]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [9]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [10]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  24347.149253731346
Tolerance for the Iteration  1 :  5738.047340760984
Tolerance for the Iteration  2 :  4233.401207243461
Tolerance for the Iteration  3 :  3053.286773212517
Tolerance for the Iteration  4 :  2260.020994575045
Tolerance for the Iteration  5 :  2071.9663072077083
Tolerance for the Iteration  6 :  1332.8198378882962
Tolerance for the Iteration  7 :  930.60627026363
Tolerance for the Iteration  8 :  953.1505223322054
Tolerance for the Iteration  9 :  1113.2957639806136
Tolerance for the Iteration  10 :  905.9376575263857
Tolerance for the Iteration  11 :  869.7334801657107
Tolerance for the Iteration  12 :  637.5754923413567
Tolerance for the Iteration  13 :  789.0475380543317
Tolerance for the Iteration  14 :  562.6161364108963
Tolerance for the Iteration  15 :  508.79689209278865
Tolerance for the Iteration  16 :  422.20245952058326
Tolerance for the Iteration  17 :  437.27526928466716
Tolerance for the Iteration  18 :  500.127531990989

In [11]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [12]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25436336745.41117


In [13]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([2, 3, 7, 9, 1, 0, 8, 3, 5, 9])

In [14]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.08625

In [15]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  40695.50313339302
Tolerance for the Iteration  1 :  13917.948354561282
Tolerance for the Iteration  2 :  5815.934105099917
Tolerance for the Iteration  3 :  1425.5962568670661
Tolerance for the Iteration  4 :  1281.7922522117983
Tolerance for the Iteration  5 :  1282.9011947657277
Tolerance for the Iteration  6 :  1570.4865947464314
Tolerance for the Iteration  7 :  1758.829287488664
Tolerance for the Iteration  8 :  1928.0444562809012
Tolerance for the Iteration  9 :  1061.782825588735
Tolerance for the Iteration  10 :  908.1288600941311
Tolerance for the Iteration  11 :  290.91785341689706
Tolerance for the Iteration  12 :  0.0


In [16]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [17]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 8, 3, 1, 4, 4, 0, 5, 0, 3])

In [29]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.125

In [19]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  29005.20350109409
Tolerance for the Iteration  1 :  4946.992989433635
Tolerance for the Iteration  2 :  2392.016525036871
Tolerance for the Iteration  3 :  2079.6555588307046
Tolerance for the Iteration  4 :  1953.7509658163203
Tolerance for the Iteration  5 :  1520.6291136337577
Tolerance for the Iteration  6 :  1371.93200360604
Tolerance for the Iteration  7 :  1063.3528922427229
Tolerance for the Iteration  8 :  1147.9830196399348
Tolerance for the Iteration  9 :  1025.1703594091614
Tolerance for the Iteration  10 :  1079.6176691729322
Tolerance for the Iteration  11 :  1238.9355528652138
Tolerance for the Iteration  12 :  957.666690027849
Tolerance for the Iteration  13 :  892.8394680563185
Tolerance for the Iteration  14 :  870.8358227645856
Tolerance for the Iteration  15 :  875.8262181069956
Tolerance for the Iteration  16 :  687.4192635201576
Tolerance for the Iteration  17 :  664.972055423536
Tolerance for the Iteration  18 :  580.214464170591

In [20]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [21]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25436336745.41117
Jacard SSE: 34361687572.938736
Cosine SSE : 25561036925.939995


In [22]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([7, 8, 2, 6, 0, 1, 3, 9, 5, 4])

In [28]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.09125

In [27]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.08625
Jacard accuracy: 0.125
Cosine accuracy : 0.09125
